In [ ]:
#Version 1: Fixed kernel and n_restart_optimizer

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import WhiteKernel, Matern, RBF, DotProduct, RationalQuadratic, ExpSineSquared
np.random.seed(1)

df = pd.read_csv('kc_house_data.csv')
df.corr('pearson')
dfn = df[['bathrooms', 'sqft_living']]
dfn2 = df.iloc[20000:,]
dfn3 = dfn2[['bathrooms', 'sqft_living']]

X = dfn3
y = dfn2['price']

from sklearn.model_selection import train_test_split



for i in np.arange(0.1, 1., 0.1):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = i, random_state=101)
    kernel = 1.0**2 * Matern(length_scale=0.5, length_scale_bounds=(1e-05, 100000.0), nu=0.5)
    gp = GaussianProcessRegressor(kernel=kernel, alpha=5e-9, optimizer='fmin_l_bfgs_b', n_restarts_optimizer=0, normalize_y=False, copy_X_train=True, random_state=2016)
    gp.fit(X_train, y_train)
    y_pred, sigma = gp.predict(X_test, return_std=True)
    fig, ax = plt.subplots(figsize = (6,4))
    plt.scatter(X_test['sqft_living'], y_pred, color='red', label = 'GPR prediction')
    plt.scatter(X_test['sqft_living'], y_test, color='blue', label = 'y_test')
    sns.regplot(X_test['sqft_living'], y_test, label = 'GPR Reg_plot')
    plt.xlabel('Sqft_living')
    plt.ylabel('Price')
    plt.title('Test_size {0}'.format(i))
    plt.legend()
    plt.savefig('EMG {0}.png'.format(i))
    plt.close()
    plt.show()

In [ ]:
#Version 2: Lopping the kernel and the n_restart_optimizer

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel 
from sklearn.gaussian_process.kernels import WhiteKernel, Matern, RBF, DotProduct, RationalQuadratic, ExpSineSquared
np.random.seed(1)

df = pd.read_csv('kc_house_data.csv')
df.corr('pearson')
dfn = df[['bathrooms', 'sqft_living']]
dfn2 = df.iloc[20000:,]
dfn3 = dfn2[['bathrooms', 'sqft_living']]

X = dfn3
y = dfn2['price']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.6, random_state=101)


kernels = [1.0*RBF(length_scale=0.5, length_scale_bounds=(1e-05, 100000.0)),
           1.0*RationalQuadratic(length_scale=0.5, alpha=0.1),
           1.0*ExpSineSquared(length_scale=0.5, periodicity=3.0,
                                length_scale_bounds=(1e-05, 100000.0),
                                periodicity_bounds=(1.0, 10.0)),
           ConstantKernel(0.1, (0.01, 10.0))*(DotProduct(sigma_0=1.0, sigma_0_bounds=(0.1, 10.0)) ** 2),
           1.0**2*Matern(length_scale=0.5, length_scale_bounds=(1e-05, 100000.0),
                        nu=0.5)] 

j=np.arange(0,10,3)


for kernel in kernels:
    for i in j:
    # Specify Gaussian Process
        if kernel == 1.0 * ExpSineSquared(length_scale=0.5, periodicity=3.0,
                                length_scale_bounds=(1e-05, 100000.0),
                                periodicity_bounds=(1.0, 10.0)):
            alpha=10
        
        elif kernel == ConstantKernel(0.1, (0.01, 10.0)) * (DotProduct(sigma_0=1.0, sigma_0_bounds=(0.1, 10.0)) ** 2):
            alpha=5
        else:
            alpha=5e-9
    
        gp = GaussianProcessRegressor(kernel=kernel, alpha=alpha, optimizer='fmin_l_bfgs_b', n_restarts_optimizer=i, normalize_y=False, copy_X_train=True, random_state=2016)
        gp.fit(X_train, y_train)
        y_pred, sigma = gp.predict(X_test, return_std=True)
        fig, ax = plt.subplots(figsize = (6,4))
        plt.scatter(X_test['sqft_living'], y_pred, color='red', label = 'GPR prediction')
        plt.scatter(X_test['sqft_living'], y_test, color='blue', label = 'y_test')
        sns.regplot(X_test['sqft_living'], y_test, label = 'GPR Reg_plot')
        plt.xlabel('Sqft_living')
        plt.ylabel('Price')
        #plt.title("(kernel:  %s)" % kernel i)
        plt.title("Kernel {0}".format(kernel))
        plt.suptitle("N_restarts {0}".format(i))
        plt.legend()
        #plt.savefig('EHG {0}.png'.format(i))
        #plt.close()
        plt.show()